# **Método dos Mínimos Quadrados**
---
<ul>
  <li><a href="#scrollTo=6T7Gy-Docn5C&uniqifier=1">Mínimos Quadrados</a>
  </li>
  <ul>
      <li><a href="#scrollTo=E2wcnL1ak8bG&uniqifier=1">Introdução</a></li>
    <li><a href="#scrollTo=pFFXXiPXkxvA&uniqifier=1">Funções comuns</a></li>
    <li>Aproximação Polinomial
      <ul>
          <li><a href="#scrollTo=4Vd2bqLKpwpr&uniqifier=1">Caso Contínuo</a></li>
          <li><a href="#scrollTo=JpN1O-_mHgEp&uniqifier=1">Caso Discreto</a></li>
      </ul>
    </li>
    <li><a href="#scrollTo=0f95KhC9xzad&uniqifier=1">Aplicações</a></li>
  </ul>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import math as m
import timeit

## **Introdução**

Dado $f$:

Seja $f$ uma função qualquer, e $V$ um espaço de funções conhecidas, gerado pelas funções {$\varphi_0$ , $\dots$ , $\varphi_n$}.

Queremos encontrar uma combinação de funções conhecidas $F$, que aproxime-se de $f$, ou seja, que minimize a distância entre $f$ e $F$.

Sendo $F$ = $\alpha_0$$\varphi_0$ + $\dots$ + $\alpha_n$$\varphi_n$.

Para encontrar $F$, precisamos resolver o seguinte sistema:

$\begin{bmatrix}
  [\varphi_0, \varphi_0] & 
    \dots & 
    [\varphi_0, \varphi_n] \\[1ex]
  \vdots & 
     & 
    \vdots \\[1ex]
  [\varphi_n, \varphi_0] & 
    \dots & 
    [\varphi_n, \varphi_n]
\end{bmatrix}$
$\begin{bmatrix}
  \alpha_0 \\[1ex]
  \vdots \\[1ex]
  \alpha_n
\end{bmatrix}$ = 
$\begin{bmatrix}
  [f, \varphi_0] \\[1ex]
  \vdots \\[1ex]
  [f, \varphi_n]
\end{bmatrix}$

Para encontrar $F$ vamos utilizar a Aproximação Polinomial.

## Decorator para medir tempo.

In [2]:
import time

# Decorator timeit
# Retorna o tempo de excução da função
# Não modifica a função.
# Para mais informações sobre decorators e timeit:
# https://medium.com/pythonhive/python-decorator-to-measure-the-execution-time-of-methods-fa04cb6bb36d
def timeit(method):
    def timed(*args, **kw):
        ts = time.time();
        result = method(*args, **kw);

        te = time.time();

        # Armazenar resultado
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__);
            kw['log_time'][name].append((te - ts) * 1000);
        else:
            print("%r  %2.5f ms" % (method.__name__, (te - ts) * 1000));
        return result
    return timed

## Aproximação Polinomial

### Caso Contínuo

No caso discreto, queremos aproximar a função $f$ de um polinômio $P_m$:

$f(x) ≅ P_m(x) = \alpha_0 + \alpha_1x + \dots + \alpha_mx^m$

Como sabemos que $B$ = {$1, x, x^2, \dots, x^m$} é base de $P_m$, temos que resolver os seguinte sistema linear para encontrar $P_m(X)$:

$\begin{bmatrix}
  [1, 1] &
    [1, x] &
    \dots & 
    [1, x^m] \\[1ex]
  [x, 1] &
    [x, x] &
    \dots & 
    [x, x^m] \\[1ex]
  \vdots & 
    \vdots &
    \dots &
    \vdots \\[1ex]
  [x^m, 1] &
    [x^m, x] &
    \dots & 
    [x^m, x^m]
\end{bmatrix}$
$\begin{bmatrix}
  \alpha_0 \\[1ex]
  \alpha_1 \\[1ex]
  \vdots \\[1ex]
  \alpha_m
\end{bmatrix}$ = 
$\begin{bmatrix}
  [1, f] \\[1ex]
  [x, f] \\[1ex]
  \vdots \\[1ex]
  [x^m, f]
\end{bmatrix}$

Sendo $[a, b]$ = $\int_{0}^{1}a(x)b(x)dx$

In [95]:
def Cholesky(M):
    M_copy = np.tril(M)
    row = M.shape[0]
    
    for x in range(row):
        for y in range(x+1):
            sum_ = 0
            for z in range(y):
                sum_ = sum_ + M_copy[x][z]*M_copy[y][z]
            if(x == y):
                M_copy[x][y] = m.sqrt(M_copy[x][y]-sum_)
            else:
                M_copy[x][y] = (1/M_copy[y][y]*(M_copy[x][y]-sum_))
    return M_copy

def lower(L, N):
    row = L.shape[0]
    z = np.zeros(row)
    
    for x in range(row):
        z[x] = (N[x]-L[x,0:x].dot(z[0:x]))/L[x,x]
    
    return z

def upper(U, z):
    row = U.shape[0]
    alpha = np.zeros(row)
    
    for x in range(row-1, -1, -1):
        alpha[x] = (z[x]-U[x,x+1:n].dot(alpha[x+1:n]))/U[x,x]
        
    return alpha

#M * alpha = N
def continuous(M, N):
    #Fatoração de Cholesky de M
    new_M = Cholesky(M)
    
    #Calculando alpha
    z = lower(new_M, N)
    alpha = upper(np.transpose(new_M), z)
    
    return alpha

### Caso Discreto

No caso discreto, conhecemos (n+1) pares de pontos que pertencem a função $f$, então consideramos:

$y_i = f(x_i)$

Sendo $(x_i, y_i)$ os pontos conhecidos, e i = 0, 1, ..., n-1, n.

Assim como no caso contínuo, queremos aproximar a função $f$ de um polinômio $P_m$:

$f(x) ≅ P_m(x) = α_0 + α_1x + \dots + α_mx^m$, m < n


Sabendo que:

$P_m(x_0) = \alpha_0 + \alpha_1x_0 + \alpha_2x_0^2 + \dots + \alpha_mx_0^m$

$P_m(x_1) = \alpha_0 + \alpha_1x_1 + \alpha_2x_1^2 + \dots + \alpha_mx_1^m$

$\vdots$

$P_m(x_0) = \alpha_0 + \alpha_1x_0 + \alpha_2x_0^2 + \dots + \alpha_mx_0^m$

e supondo que $p = (P_m(x_0), P_m(x_1), \dots, P_m(x_n))$

Temos que:

$p = \alpha_0$
$\begin{bmatrix}
  1 \\[1ex]
  1 \\[1ex]
  \vdots \\[1ex]
  1
\end{bmatrix}$ +
 $\alpha_1$
$\begin{bmatrix}
  x_0 \\[1ex]
  x_1 \\[1ex]
  \vdots \\[1ex]
  x_n
\end{bmatrix}$ +
 $\alpha_2$
$\begin{bmatrix}
  x_0^2 \\[1ex]
  x_1^2 \\[1ex]
  \vdots \\[1ex]
  x_n^2
\end{bmatrix}$ +
$\dots$ +
$\alpha_m$
$\begin{bmatrix}
  x_0^m \\[1ex]
  x_1^m \\[1ex]
  \vdots \\[1ex]
  x_n^m
\end{bmatrix}$

Supondo que $Y = (y_0, y_1, \dots, y_n)$

E sabendo que o conjunto $U = (u_0, u_1, \dots, u_n)$ é base. Sendo:

$u_0 = [1, 1, \dots, 1]$

$u_1 = [x_0, x_1, \dots, x_n]$

$u_2 = [x_0^2, x_1^2, \dots, x_n^2]$

$\vdots$

$u_m = [x_0^m, x_1^m, \dots, x_n^m]$

Temos que resolver o seguinte sistema linear para encontrar $P_m(x)$:

$\begin{bmatrix}
  [u_0, u_0] &
    [u_0, u_1] &
    \dots & 
    [u_0, u_m] \\[1ex]
  [u_1, u_0] &
    [u_1, u_1] &
    \dots & 
    [u_1, u_m] \\[1ex]
  \vdots & 
    \vdots &
    \dots &
    \vdots \\[1ex]
  [u_m, u_0] &
    [u_m, u_1] &
    \dots & 
    [u_m, u_m]
\end{bmatrix}$
$\begin{bmatrix}
  \alpha_0 \\[1ex]
  \alpha_1 \\[1ex]
  \vdots \\[1ex]
  \alpha_m
\end{bmatrix}$ = 
$\begin{bmatrix}
  [u_0, Y] \\[1ex]
  [u_1, Y] \\[1ex]
  \vdots \\[1ex]
  [u_m, Y]
\end{bmatrix}$

Sendo $[a, b]$ = $\sum_{k=0}^{n}a_kb_k$

In [7]:
# @TODO: Código para Caso Discreto

# Aplicações

In [8]:
# @TODO: Aplicações dos códigos